<img src='h1.png'></img>

<img src='h2.png'></img>

In [1]:
import numpy as np
from scipy.special import binom as C

In [46]:
M = 100 #число хромосом
mu = 100000 #частота мутаций
K = 3 #количество ответвлений, считая от изначального предка к конечным потомкам = количество конечных потомков

In [4]:
T = np.array([0, .04, .01, .07], dtype=float) #периоды времени
N = np.array([2, 5, 10, 6], dtype=int) #размеры популяции

In [5]:
def v(t):
    '''
    размер популяции в данный момент времени t
    '''
    cond = list()
    for i in range(len(T) - 1):
        cond.append(T[i] <= t < T[i + 1])
    cond.append(t >= T[-1])
    return np.piecewise(t, cond, N)

In [6]:
def w(K, t):
    '''
    интенсивность процесса Пуассона
    K - количество конечных потомков
    t - время
    '''
    return K * mu + 1 / v(t) * C(K, 2)

In [7]:
def add_chromosome(lineage, pos, mutated):
    '''
    добавление хромосомы к особи и её потомкам
    lineage - ветка предок-потомки
    pos - позиция в геноме - хромосома, которую мы хотим добавить
    mutated - признак мутации: 1 - есть, 0 - нет
    '''
    #заходим в нулевой элемент списка - это вложенный список. В нём первый эемелент - это словарь.
    lineage[0][1][pos] = mutated
    #если список lineage содержит потомка или двоих, повторяем действие
    for i in range(1, len(lineage)):
        add_chromosome(lineage[i], pos, mutated)

In [23]:
def simulation(K, mu, T, N):
    '''
    коалесцентная модель
    K - количество конечных потомков
    mu - частота мутаций
    T - периоды времени
    N - размеры популяции
    
    возвращает дерево в виде вложенных списков и времена Пуассоновского процесса для мутации и коалесценции
    '''
    #особь, хромосомы, время коалесценции, количество мутаций
    decendents = list([[i, {}, .0, 0]] for i in range(K)) #основа будущего дерева
    #np.random.seed(1)
    t = 0
    
    while K > 1:
        W1 = w(K, t)
        '''
        генерация времени до следующего события Пуассоновского процесса
        время имеет экспоненциальное распределение
        параметр равен интенсивности Пуассоновского процесса, т.е. w(K,t)
        '''
        t += np.random.exponential(1 / W1)
        #Проверка
        W2 = w(K, t)
        if W1 != W2:
            for tau in T:
                if w(K, tau) == W2:
                    t = tau
                    break
        W = w(K, t)
        
        weight = K * mu
        
        '''
        генерация события мутации, используя распределение Бернулли
        вес weight = K * mu
        вероятность weight / w(K, t)
        
        используется биномиальное распределение с числом испытаний = 1,
        поэтому количество успехов может быть равно либо 1, либо 0
        
        из определения распределения Бернулли неудача события мутации даёт успех события коалесценции 
        '''
        if np.random.binomial(1, weight / W):
            #случайно выбираем одного потомка, который оказался подвержен мутации
            l = np.random.randint(0, K)
            #случайно выбираем хромосому, которая оказалась подвержена мутации
            p = np.random.random()
            decendents[l][0][3] += 1
            #всем потомкам особи l передаём эту мутацию - работем внутри ветки decendents[l]
            add_chromosome(decendents[l], p , 1)
            #остальные ветки получают ноль
            for i in range(l):
                add_chromosome(decendents[i], p, 0)
            for i in range(l + 1, K):
                add_chromosome(decendents[i], p, 0)
            '''
            print('K =', K)
            print('t =', t)
            print('мутация хромосомы', p, 'у особи', l)
            print('--------------------------------')
            '''
        else:
            l1, l2 = np.random.choice(K, 2, replace=False)
            lmin = min(l1, l2) 
            lmax = max(l1, l2)
            '''
            print('K =', K)
            print('t =', t)
            print('коалесценция между особями', l1, 'и', l2)
            print('--------------------------------')
            '''
            K -= 1
            ancestors = list([[i, {}, t, 0]] for i in range(K))
            for i in range(lmin):
                ancestors[i].append(decendents[i])
            #чтобы провести даньнейшую связь предков и потомков, проведём замену
            decendents[lmin + 1], decendents[lmax] = decendents[lmax], decendents[lmin + 1]
            ancestors[lmin].append(decendents[lmin])
            ancestors[lmin].append(decendents[lmin + 1])
            for i in range(lmin + 1, K):
                ancestors[i].append(decendents[i + 1])
            decendents = ancestors
            
    return decendents

In [47]:
tree = simulation(K, mu, T, N)

Проверка на равномерность распределения мутации

In [48]:
length1 = abs(tree[0][1][1][0][2] - tree[0][1][0][2])

In [49]:
length2 = abs(tree[0][0][2] - tree[0][1][0][2])

In [50]:
mut1 = tree[0][1][1][0][3]
mut2 = tree[0][1][0][3]

In [51]:
mut1, mut2

(167601, 165488)

In [52]:
abs(length1 / mut1 - length2 / mut2)

1.4684646461891936e-07

### Коалесценция
На каждом поколении своя нумерация. Принадлежность потомка непосредственному предку обозначается их нахождением в одном списке, причём потомок имеет на один уровень вложенности больше.

Например:

[0, [1] ] - предок 0 со своим потомком 1.

[0, [1, [1], [2] ] ] - предок 0 со своим потомком 1, у которого потомки 1 и 2.

### Добавляем мутацию
Теперь каждая особь это не число, а список из 4 элементов. Первый элемент - особь. Второй - словарь. Ключи словаря - хромосомы. Значение каждого ключа - 0 или 1 - наличие мутации у данной хромосомы. Третий элемент - время коалесценции. Четвёртый - количество мутаций.

Например:

[[0, {}, t, 0], [[1, {0.5: 1}, 0, 1]] ] - предок 0 со своим потомком 1. У потомка мутация хромосомы 0.5

[[0, {}, t, 0], 

    [[1, {0.5: 1}, t1, 1], 
        [[1, {0.5: 1, 0.67: 0}, 0, 0]], 
        [[2, {0.5: 1, 0.67: 1}, 0, 1]] 
    ] 
]

предок 0 со своим потомком 1 (мутация 0.5), у которого потомки 1 (мутация 0.5; 0.67 без мутации) и 2 (мутация 0.5 и 0.67).